In [0]:
# imports
import re
import numpy as np
import pandas as pd
from pprint import pprint
import pickle

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# Plotting tools
#!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


In [0]:
pickle = pd.read_pickle('topic_lemmas.pkl')
pickle = list(filter(lambda x: x != None, pickle))
lemmatize_title = lambda dic: dic['JJ'] + dic['NN'] + dic['NNP'] + dic['NNT'] + dic['VB']
lemmatized_titles = [lemmatize_title(title) for title in pickle]

In [0]:
# Imports from github/ido90 source
from hebrew_stopwords import hebrew_stopwords
from Parser import tokenize
is_stopword = lambda word: bool(word in hebrew_stopwords)
is_jibbrish = lambda word: bool(len(word) < 3)
is_a_year = lambda word: (word == "תשח") or (word == "תשע") or (word == "תשס")
tests = [is_stopword, is_jibbrish, is_a_year]
filter_word = lambda word : sum([test(word) for test in tests]) == 0
tokenized_titles = [list(filter(filter_word, title)) for title in lemmatized_titles]

In [0]:
titles = tokenized_titles

# Create Dictionary
id2word = corpora.Dictionary(titles)

# Create Corpus
corpus = [id2word.doc2bow(title) for title in titles]

In [61]:
from gensim.models import TfidfModel
from gensim.corpora import Dictionary
tfidf_model = TfidfModel(corpus)
for i in range(20):
  doc = corpus[i]
  for (word_id, tf_idf) in tfidf_model[doc]:
    pprint(str((id2word[word_id], tf_idf)))
  a = input("hi")


KeyboardInterrupt: ignored

In [65]:
def calc_model_and_score(num_topics):  
  # Build LDA model
  lda_model = gensim.models.ldamodel.LdaModel(corpus = corpus,
                                             id2word = id2word,
                                             num_topics = num_topics,
                                             random_state = 100,
                                             passes = 10,
                                             alpha = 'auto',
                                             per_word_topics = True)
  coherence_model_ldamallet = CoherenceModel(model=lda_model, texts=titles, dictionary=id2word, coherence='c_v')
  coherence_ldamallet = coherence_model_ldamallet.get_coherence()
  return lda_model, coherence_ldamallet

/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:804: RuntimeWarning: divide by zero encountered in double_scalars
  perwordbound = self.bound(chunk, subsample_ratio=subsample_ratio) / (subsample_ratio * corpus_words)


(<gensim.models.ldamodel.LdaModel at 0x7f3957908748>, 0.567612193440552)

In [69]:
data = {i : calc_model_and_score(i) for i in [20, 50, 80, 100, 140, 170, 200]}
df = pd.DataFrame.from_dict(data, orient='index')
df.to_pickle('results')

/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:804: RuntimeWarning: divide by zero encountered in double_scalars
  perwordbound = self.bound(chunk, subsample_ratio=subsample_ratio) / (subsample_ratio * corpus_words)


In [0]:
# Print the Keyword in the 10 topics
# pprint(lda_model.print_topics())
# doc_lda = lda_model[corpus]

[(8,
  '0.127*"בטיפול" + 0.089*"סקירה" + 0.081*"בתקופת" + 0.042*"שכונת" + '
  '0.038*"סרטן" + 0.036*"נפש" + 0.030*"הגליל" + 0.024*"הנמלים" + '
  '0.023*"למינוי" + 0.020*"ישובים"'),
 (22,
  '0.080*"תקשורת" + 0.072*"בגין" + 0.058*"מחדש" + 0.054*"פיצוי" + 0.049*"זמן" '
  '+ 0.043*"השקעת" + 0.042*"הצבעה" + 0.037*"איחוד" + 0.037*"בהצח" + '
  '0.031*"תוצרת"'),
 (60,
  '0.154*"היערכות" + 0.100*"ציבורי" + 0.054*"צמצום" + 0.049*"במקום" + '
  '0.049*"התנועה" + 0.045*"העולמית" + 0.044*"אדמה" + 0.032*"סמכות" + '
  '0.030*"הדרכים" + 0.022*"בתאונות"'),
 (82,
  '0.147*"לקידום" + 0.137*"מעמד" + 0.067*"ליקויים" + 0.055*"האינטרנט" + '
  '0.051*"האישה" + 0.042*"כתוצאה" + 0.040*"לציבור" + 0.031*"מפגש" + '
  '0.030*"אלפי" + 0.029*"בעובדים"'),
 (12,
  '0.227*"הסדרת" + 0.100*"מועד" + 0.091*"העיסוק" + 0.053*"סטודנטים" + '
  '0.050*"צער" + 0.045*"הדוח" + 0.035*"הגנה" + 0.034*"חשבונות" + 0.032*"עלידי" '
  '+ 0.022*"יועצי"'),
 (33,
  '0.200*"דין" + 0.172*"המשטרה" + 0.058*"מערכות" + 0.027*"רציפות" + 0.024*"כח" '


In [0]:
# Visualize
# pyLDAvis.enable_notebook()
# vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
#  vis

KeyboardInterrupt: ignored